# Monitoring artisanal fishing activity in Gunjur Bay (The Gambia)

This notebook is a transparent, reproducible workflow to **detect and count artisanal fishing boats** in Gunjur Bay (The Gambia) using ~3 m PlanetScope surface reflectance imagery.

Why:
- Coastal communities in southern Gambia depend on small-scale fishing.
- Fishmeal / fish oil (FMFO) factories operating nearby have been accused of over-extracting fish for export.
- We want independent, open monitoring of fishing effort: *How many boats are in the water? When?*

What this notebook does:
1. Loads a PlanetScope scene (multispectral GeoTIFF).
2. Builds a true-color RGB preview.
3. Computes NDWI (Normalized Difference Water Index) to separate water from everything else.
4. Creates a binary "water mask" using an NDWI threshold that **you choose**.
5. Cleans that mask and clips it to the bay polygon (region of interest).
6. Treats "holes in water" as candidate boats, labels them, filters by size, and counts them.
7. Draws bounding boxes so a human can visually confirm detections.

What this notebook is **not**:
- It's not estimating biomass, catch volume, or fishing pressure in tons. A higher boat count is a proxy for fishing effort — it's not a direct measure of fish extraction.
- It's not a deep learning model. It's deliberately simple and auditable.

Assumptions / caveats:
- Works best on calm water, low turbidity, low sunglint.
- Boats that raft together may merge into one detection - therefore estimates are conservative.
- Very foamy surf / breaking waves can look like small "non-water" blobs and inflate counts.
- You **must** pick the NDWI threshold manually per scene (details below). Different dates and lighting need different thresholds.

---
### Workflow
1. Compute NDWI.
2. Pick a threshold that makes open water = white and boats = dark.
3. Clean the water mask.
4. Clip to the bay polygon.
5. Connected components ⇒ boat blobs ⇒ count.


## 1. Imports and configuration

In this section:
- Import all libraries.
- Set file paths.
- Set parameters like `threshold_for_ndwi`, size filters for boats, etc.

**IMPORTANT:**
- You *will* have to tune `threshold_for_ndwi` for each new scene. There is no universal magic number.
- The logic for that is explained again in Section 4.

Dependencies you need installed (conda / pip):
- `numpy`
- `matplotlib`
- `scipy`
- `gdal` / `osgeo`
- `rasterio`
- `geopandas` (optional helper)
- `shapely`

Make sure `gdal` and `rasterio` are installed in the same environment so CRS handling behaves.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from osgeo import gdal, gdalconst, ogr
from scipy import stats
from scipy.ndimage import (
    generate_binary_structure,
    binary_closing,
    binary_opening,
    label,
    find_objects,
    sum as nd_sum
)
from mpl_toolkits.axes_grid1 import make_axes_locatable

import rasterio
from rasterio.transform import Affine
from rasterio.crs import CRS

import geopandas as gpd  # not strictly required for rasterization, but useful context
from shapely.geometry import Polygon

# Configure matplotlib for efficient PNG output
matplotlib.rcParams['figure.dpi'] = 150
matplotlib.rcParams['savefig.dpi'] = 150
matplotlib.rcParams['savefig.format'] = 'png'
matplotlib.rcParams['savefig.bbox'] = 'tight'
# Optimize PNG compression
matplotlib.rcParams['savefig.pil_kwargs'] = {'optimize': True, 'compress_level': 9}

# -----------------------
# User configuration
# -----------------------

# Path to the PlanetScope surface reflectance scene (multispectral GeoTIFF)
# The script assumes either 4-band (B,G,R,NIR) or 8-band PSScene product.
filename_tif = "data/your_planetscope_scene.tif"

# Path to the shapefile (or any vector) describing the bay / region of interest
filename_shp = "data/region_of_interest.shp"

# Directory for all outputs (PNGs, GeoTIFF masks, etc.)
out_dir = "results/"

# Visual padding in plots, just so boxes don't hug the array edges
figure_border = 25

# Tiny constant to keep us from dividing by zero when we compute NDWI
epsilon = 1e-4

# =============================
# NDWI threshold FOR THIS SCENE
# =============================
# You MUST choose this manually.
# The workflow is: try 0.10, 0.14, 0.18, ... inspect the mask visually,
# keep the one that best marks "open water" as white (1) and boats as dark (0).
# There is no single global answer because water color, turbidity, and sun angle change.
#
# Future work: this could be automated by looking at the NDWI histogram
# and picking a split, but we're not doing that here.

threshold_for_ndwi = 0.14  # <-- TUNE ME IN STEP 4

# Connected-component size filters (in pixels) for what "counts" as a boat
# Note: 1 pixel ~ (3 m)^2 for PlanetScope, so these bounds imply a plausible boat footprint
min_size = 6      # below this is usually foam / tiny noise
max_size = 300    # above this is usually pier shadows / merged clusters / junk


## 2. Read imagery and build RGB

Here we:
1. Open the PlanetScope raster.
2. Detect whether it's a 4-band or 8-band product.
3. Extract BLUE, GREEN, RED, and NIR.
4. Create a stretched RGB image for human viewing.
5. Save that RGB as both a GeoTIFF and a PNG.

This RGB is mostly for reporting / map figures later. It's not strictly needed for detection, but it's very helpful for visual confirmation.

PlanetScope band mapping assumptions:
- 4-band SR: 1=Blue, 2=Green, 3=Red, 4=NIR
- 8-band SR: 2=Blue, 4=Green, 6=Red, 8=NIR (typical mapping; adjust if your imagery docs say otherwise)

If your band order is different, fix it here.

In [ ]:
# Open raster
dataset = gdal.Open(filename_tif, gdalconst.GA_ReadOnly)
if dataset is None:
    raise RuntimeError(f"Could not open {filename_tif}")

rows = dataset.RasterYSize
cols = dataset.RasterXSize
bands = dataset.RasterCount

# Grab georeferencing info so we can reuse it when saving derived rasters later
geotransform = dataset.GetGeoTransform()
proj_wkt = dataset.GetProjection()

# Decide which bands correspond to Blue, Green, Red, NIR
if bands >= 8:
    band_blue, band_green, band_red, band_nir = 2, 4, 6, 8
    print("Using bands 2,4,6,8 for B,G,R,NIR (8-band product)")
elif bands >= 4:
    band_blue, band_green, band_red, band_nir = 1, 2, 3, 4
    print("Using bands 1,2,3,4 for B,G,R,NIR (4-band product)")
else:
    raise ValueError(f"Need at least 4 bands, found {bands}")

# Read bands as float arrays (so math doesn't clip)
blue  = dataset.GetRasterBand(band_blue ).ReadAsArray().astype(float)
green = dataset.GetRasterBand(band_green).ReadAsArray().astype(float)
red   = dataset.GetRasterBand(band_red  ).ReadAsArray().astype(float)
nir   = dataset.GetRasterBand(band_nir  ).ReadAsArray().astype(float)

# Helper: stretch to 0-255 uint8 for visualization
def stretch_to_uint8(arr, pmin=1, pmax=99):
    a_min, a_max = np.percentile(arr, (pmin, pmax))
    scaled = np.clip((arr - a_min) / (a_max - a_min + 1e-9), 0, 1)
    return (scaled * 255).astype(np.uint8)

rgb_stack = np.dstack([
    stretch_to_uint8(red),
    stretch_to_uint8(green),
    stretch_to_uint8(blue)
])

# Save true-color composite as GeoTIFF (uint8, 3 bands)
with rasterio.open(
    out_dir + "rgb_truecolor.tif",
    "w",
    driver="GTiff",
    height=rgb_stack.shape[0],
    width=rgb_stack.shape[1],
    count=3,
    dtype=rgb_stack.dtype,
    crs=CRS.from_wkt(proj_wkt),
    transform=Affine.from_gdal(*geotransform)
) as dst:
    dst.write(rgb_stack[..., 0], 1)
    dst.write(rgb_stack[..., 1], 2)
    dst.write(rgb_stack[..., 2], 3)

# Quicklook PNG for publication
plt.figure(figsize=(10,6))
plt.imshow(rgb_stack)
plt.title("True-color composite (contrast-stretched)")
plt.axis("off")
plt.savefig(out_dir + "00_rgb_truecolor.png", bbox_inches="tight")
plt.show()


## 3. Compute NDWI

**Normalized Difference Water Index (NDWI)** helps us tell water from everything else.

$$\text{NDWI} = \frac{G - NIR}{G + NIR + \epsilon}$$

- Water: tends to have **high NDWI**.
- Boats / structures / shoreline / glinty stuff: usually **lower NDWI**.

We'll compute NDWI, save it as a float GeoTIFF, and also make a preview PNG with a grayscale colormap plus a colorbar.


In [ ]:
# Compute NDWI
denom = (green + nir + epsilon)
ndwi = (green - nir) / denom

# Save NDWI as 32-bit float GeoTIFF
with rasterio.open(
    out_dir + "ndwi.tif",
    "w",
    driver="GTiff",
    height=ndwi.shape[0],
    width=ndwi.shape[1],
    count=1,
    dtype="float32",
    crs=CRS.from_wkt(proj_wkt),
    transform=Affine.from_gdal(*geotransform)
) as dst:
    dst.write(ndwi.astype("float32"), 1)

# Plot NDWI with nice colorbar placement
fig, ax = plt.subplots(figsize=(10,6))
im = ax.imshow(ndwi, cmap="gray")
ax.set_title("NDWI", fontsize=12)
ax.set_xlim([0 - figure_border, cols + figure_border])
ax.set_ylim([rows + figure_border, 0 - figure_border])

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax, label="NDWI value")

plt.savefig(out_dir + "01_ndwi.png", bbox_inches="tight")
plt.show()


## 4. Pick the NDWI threshold and build the water mask

The next step is to turn NDWI (continuous values) into a simple **binary water mask**.

**Concept**
- We call a pixel "water" if `NDWI > threshold`.
- Everything else becomes "not water".

Why this matters:
- Once we have a clean water mask, small dark holes *inside that water* are good candidates for boats.
- If the mask is too aggressive or too loose, boats will either disappear into the water or the entire surf zone turns into "water" and floods the bay.

### VERY IMPORTANT: the threshold is **not fixed**
- There is **no universal** `threshold_for_ndwi` that works for all images.
- Water color changes with turbidity, algae, sediment plumes, sun glint, haze, etc.
- So you must choose the threshold **per scene**.

### How to choose it (slider below)
1. Drag the slider.
2. Watch the preview mask:
   - White = what we consider water.
   - Dark holes *inside that white water* = possible boats.
3. You're aiming for:
   - Open water in the bay is mostly solid white.
   - Boats show up as little dark cut-outs (not filled in white).
   - Shoreline / foam isn't completely white, otherwise you'll count waves/shoals as "water".

Behind the scenes:
- Every time you move the slider, we update a global variable called `chosen_threshold_for_ndwi`.
- The next cell ("Finalize water mask") will automatically grab that last value and make the official mask we use for the rest of the workflow.

Future improvement:
- We *could* automate picking this threshold by looking at the NDWI histogram and splitting water vs not-water automatically.
- We are **not** doing that here on purpose — we want the method to stay simple, explainable, and easy to audit.


In [6]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import matplotlib.pyplot as plt

# We'll keep the most recent slider value here so the next cell can read it
chosen_threshold_for_ndwi = None

# Safety check so we don't crash if ndwi isn't defined yet
if "ndwi" not in globals():
    raise RuntimeError("You need to run the NDWI cell (Section 3) before running the slider cell.")

def preview_water_mask(threshold):
    """
    This runs every time you move the slider.
    It:
    - builds a temporary preview mask at the given threshold
    - updates the global chosen_threshold_for_ndwi
    - plots that preview
    - prints the currently selected threshold
    """
    global chosen_threshold_for_ndwi
    chosen_threshold_for_ndwi = float(threshold)

    water_mask_preview = ndwi > threshold

    plt.figure(figsize=(10,6), dpi=200)
    plt.imshow(water_mask_preview, cmap="gray")
    plt.title(
        f"Binary water mask preview (NDWI > {threshold:.3f})\n"
        "White = water  |  Dark holes in water = possible boats",
        fontsize=11
    )
    plt.xlim([0 - figure_border, cols + figure_border])
    plt.ylim([rows + figure_border, 0 - figure_border])
    plt.axis("off")
    plt.show()

    print(
        f"Current chosen_threshold_for_ndwi = {threshold:.3f}\n"
        "This value will be used in the next cell to build the FINAL water_mask."
    )

# Slider config:
slider = widgets.FloatSlider(
    value=threshold_for_ndwi,  # initial guess from your config
    min=0.05,
    max=0.30,
    step=0.005,
    description='NDWI thr',
    continuous_update=True,
    readout_format='.3f',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

interactive_plot = widgets.interactive_output(
    preview_water_mask,
    {'threshold': slider}
)

display(
    widgets.VBox([
        widgets.HTML(
            "<b>1. Drag the slider.</b> White = water.<br>"
            "Dark holes in water = possible boats.<br>"
            "<b>2. The most recent slider value is stored automatically and will be used below.</b>"
        ),
        slider,
        interactive_plot
    ])
)


In [ ]:
# Pick the threshold to commit:
# - Prefer the last slider position (chosen_threshold_for_ndwi)
# - Otherwise fall back to the static threshold_for_ndwi from the config cell at the top
final_threshold = chosen_threshold_for_ndwi if "chosen_threshold_for_ndwi" in globals() and chosen_threshold_for_ndwi is not None else threshold_for_ndwi

print(f"Using {final_threshold:.3f} as the FINAL NDWI threshold to create water_mask.")

# Build the official water_mask using that final threshold
water_mask = ndwi > final_threshold

# Save binary water mask as uint8 GeoTIFF (1 = water, 0 = not water)
with rasterio.open(
    out_dir + "02_water_mask.tif",
    "w",
    driver="GTiff",
    height=water_mask.shape[0],
    width=water_mask.shape[1],
    count=1,
    dtype="uint8",
    crs=CRS.from_wkt(proj_wkt),
    transform=Affine.from_gdal(*geotransform)
) as dst:
    dst.write(water_mask.astype("uint8"), 1)

# plot of the committed mask
plt.figure(figsize=(10,6))
plt.imshow(water_mask, cmap="gray")
plt.title(
    f"FINAL water mask (NDWI > {final_threshold:.3f})\n"
    "This version will be used downstream.",
    fontsize=12
)
plt.xlim([0 - figure_border, cols + figure_border])
plt.ylim([rows + figure_border, 0 - figure_border])
plt.savefig(out_dir + "02_water_mask.png", bbox_inches="tight")
plt.show()


## 5. Clean the water mask (denoise)

The raw binary mask can be speckly: single-pixel noise, tiny holes, foam, etc.

We'll do:
1. **3×3 mode filter**: replace each pixel with the most common value in its 3×3 neighborhood. This kills salt-and-pepper noise without chewing up boats.
2. (Optional, commented out) a gentle morphological close/open. This can fill pinholes and smooth edges, but it *can* erase tiny boats, so it's off by default.

Result: a cleaner water mask where "ocean water" is nice and solid.

In [ ]:
# Apply 3x3 mode filter
rows, cols = water_mask.shape
mode_filtered_mask = np.copy(water_mask)
for i in range(1, rows - 1):
    for j in range(1, cols - 1):
        window = water_mask[i-1:i+2, j-1:j+2]
        mode_filtered_mask[i, j] = stats.mode(
            window.ravel().astype(int),
            keepdims=True
        )[0][0]

cleaned_water_mask = mode_filtered_mask


# Save cleaned mask
with rasterio.open(
    out_dir + "03_cleaned_water_mask.tif",
    "w",
    driver="GTiff",
    height=cleaned_water_mask.shape[0],
    width=cleaned_water_mask.shape[1],
    count=1,
    dtype="uint8",
    crs=CRS.from_wkt(proj_wkt),
    transform=Affine.from_gdal(*geotransform)
) as dst:
    dst.write(cleaned_water_mask.astype("uint8"), 1)

# plot
plt.figure(figsize=(10,6))
plt.imshow(cleaned_water_mask, cmap="gray")
plt.title("Cleaned water mask", fontsize=12)
plt.xlim([0 - figure_border, cols + figure_border])
plt.ylim([rows + figure_border, 0 - figure_border])
plt.savefig(out_dir + "03_cleaned_water_mask.png", bbox_inches="tight")
plt.show()


## 6. Clip to study area polygon (ROI)

We only care about vessels *inside* our bay / study area. We'll:
1. Rasterize the study-area polygon (e.g. Gunjur Bay outline) onto the same grid as the PlanetScope image.
2. Combine that with the cleaned water mask: `roi_water_mask = cleaned_water_mask & region_mask`.

That prevents surf or weird reflections far away from messing with the boat count.

In [9]:
# Rasterize the shapefile polygon to match our raster grid
shp_ds = ogr.Open(filename_shp)
if shp_ds is None:
    raise RuntimeError(f"Could not open {filename_shp}")
layer = shp_ds.GetLayer()

mem_driver = gdal.GetDriverByName("MEM")
region_ds = mem_driver.Create("", cols, rows, 1, gdal.GDT_Byte)
region_ds.SetGeoTransform(geotransform)
region_ds.SetProjection(proj_wkt)

gdal.RasterizeLayer(region_ds, [1], layer, burn_values=[1])
region_mask = region_ds.ReadAsArray().astype(bool)


# Combine polygon with cleaned water mask
roi_water_mask = cleaned_water_mask & region_mask

# Save ROI water mask
with rasterio.open(
    out_dir + "05_roi_water_mask.tif",
    "w",
    driver="GTiff",
    height=roi_water_mask.shape[0],
    width=roi_water_mask.shape[1],
    count=1,
    dtype="uint8",
    crs=CRS.from_wkt(proj_wkt),
    transform=Affine.from_gdal(*geotransform)
) as dst:
    dst.write(roi_water_mask.astype("uint8"), 1)

## 7. Boat detection via connected components

Idea:
- After masking, water is 1.
- A boat is basically a small "hole" (0) inside that water.

Steps:
1. Invert the ROI water mask so holes become bright.
2. Run connected-component labeling with 4-connectivity.
3. Measure each blob's area in pixels.
4. Keep blobs within `[min_size, max_size]` as plausible vessels.

Then we count them.

Caveats:
- Tiny foam splashes should get filtered out by `min_size`.
- Piers, long wakes, or merged rafts of boats might get filtered out by `max_size`.
- When boats raft together, they can merge into one blob, so we may undercount.

In [ ]:
# Invert mask so boats stand out as True
inverted_mask = ~roi_water_mask

# Label connected components (4-connectivity)
structure4 = generate_binary_structure(2, 1)
labeled_blobs, n_blobs = label(inverted_mask, structure=structure4)

# Pixel area per blob
blob_sizes = nd_sum(inverted_mask, labeled_blobs, index=range(1, n_blobs+1))

# Keep only plausible boat blobs
ship_candidates = np.zeros_like(labeled_blobs, dtype=bool)
for idx, size in enumerate(blob_sizes, start=1):
    if min_size <= size <= max_size:
        ship_candidates[labeled_blobs == idx] = True

# Save candidate mask
with rasterio.open(
    out_dir + "06_ship_candidates.tif",
    "w",
    driver="GTiff",
    height=ship_candidates.shape[0],
    width=ship_candidates.shape[1],
    count=1,
    dtype="uint8",
    crs=CRS.from_wkt(proj_wkt),
    transform=Affine.from_gdal(*geotransform)
) as dst:
    dst.write(ship_candidates.astype("uint8"), 1)

# plot of the ship candidate mask
plt.figure(figsize=(10,6))
plt.imshow(ship_candidates, cmap="gray")
plt.title("Ship candidate mask", fontsize=12)
plt.xlim([0 - figure_border, cols + figure_border])
plt.ylim([rows + figure_border, 0 - figure_border])
plt.savefig(out_dir + "06_ship_candidates.png", bbox_inches="tight")
plt.show()

# Count detections in size range
labeled_final, n_final = label(ship_candidates, structure=structure4)
slices_final = find_objects(labeled_final)

boat_count = 0
for i, slc in enumerate(slices_final, start=1):
    if slc is None:
        continue
    area_here = np.sum(labeled_final[slc] == i)
    if min_size <= area_here <= max_size:
        boat_count += 1

print(f"Estimated boat count in ROI: {boat_count}")
print("Reminder: this is per-scene, not per-day effort, and big clusters may merge.")


## 8. Visual : bounding boxes on RGB and NDWI

Now let's make human-checkable plots:
- Red rectangles around each detected blob, drawn over:
  1. The RGB composite (easy to read for non-experts)
  2. The NDWI layer (shows what the algorithm “sees”)

These PNGs are what you show in reports, talks, posts, etc. They also make it clear this is not a black box: you can literally see what was counted.

In [ ]:
def draw_boxes(base_image, cmap, title, outfile):
    fig, ax = plt.subplots(figsize=(10,6))
    if cmap:
        ax.imshow(base_image, cmap=cmap)
    else:
        ax.imshow(base_image)
    ax.set_title(title, fontsize=12)

    for i, slc in enumerate(slices_final, start=1):
        if slc is None:
            continue
        row_slice, col_slice = slc
        area_here = np.sum(labeled_final[slc] == i)
        if min_size <= area_here <= max_size:
            r0, r1 = row_slice.start, row_slice.stop
            c0, c1 = col_slice.start, col_slice.stop
            width  = c1 - c0
            height = r1 - r0

            rect = plt.Rectangle(
                (c0, r0),
                width,
                height,
                linewidth=0.4,
                edgecolor="r",
                facecolor="none"
            )
            ax.add_patch(rect)

    ax.set_xlim([0 - figure_border, cols + figure_border])
    ax.set_ylim([rows + figure_border, 0 - figure_border])
    plt.savefig(out_dir + outfile, bbox_inches="tight")
    plt.show()

draw_boxes(rgb_stack,
           cmap=None,
           title="Detected boats (red boxes) on RGB",
           outfile="07_ships_in_rgb.png")

draw_boxes(ndwi,
           cmap="gray",
           title="Detected boats (red boxes) on NDWI",
           outfile="08_ships_in_ndwi.png")


## 9. Interpretation and limitations

How to interpret the result:
- `boat_count` = number of blobs that look like boats in this specific PlanetScope scene.
- More boats usually means more active fishing at that timestamp.

But:
- This is **not** a biomass estimate.
- Foam and surf can inflate counts if they pass the size test.
- When boats raft together closely, they can merge into one blob and we undercount.
- Bad `threshold_for_ndwi` ⇒ bad results. Always sanity-check the figures.

Best practice:
- Save the PNGs (RGB and NDWI with red boxes) whenever you report a number.
- Log: image date/time, chosen `threshold_for_ndwi`, final `boat_count`.

Future work:
- Automatically choose `threshold_for_ndwi` by analyzing the NDWI histogram (instead of tuning by eye).
- Build a time series of counts to track pressure on local fish stocks.
- Ground truth with shoreline photos / eyewitness / drone when possible.

_If you share this publicly, please be explicit that this method counts visible boats as an indicator of fishing effort, not total fish taken._